<h1 style='text-align:center; text-transform: uppercase; text-decoration: underline;'>Cinemática directa</h1>
<br>
<p style='font-size:15px;'><span style='text-decoration: underline;'>Materia</span>: Proyecto en Ingeniería Mecatrónica.</p>
<p style='font-size:15px;'><span style='text-decoration: underline;'>Nombre del proyecto</span>: Robot Lúdico.</p>
<p style='font-size:15px;'><span style='text-decoration: underline;'>Alumnos</span>: Daiana Belén Viscarra Hernández & Pedro Tagliani.</p>
<br>
<p style='font-size:15px;'><span style='text-decoration: underline;'>Detalles de la versión</span>: Se consideró el marcador en el efector final.</p>
<br>

<h3>Librerías a utilizar:</h3>

In [1]:
from spatialmath import *
from spatialmath.base import *
import matplotlib.pyplot as plt
from math import pi, degrees, radians
import numpy as np
import sympy as sp
from IPython.display import display

%matplotlib qt

<h3>Cinemática Directa con algoritmo DH:</h3>

In [2]:
# Símbolos a utilizar:
q1, q2, q3, q4, l1, l2, l3, l4, l5 = sp.symbols('q1 q2 q3 q4 l1 l2 l3 l4 l5')

In [3]:
# Se extiende la SE3 para poder manejar variables en forma simbólica y numérica
class SE3_extended(SE3):
    def symbolReplace(self,symbol_values): # symbol_values --> Lista con tuplas de dos valores
        aux = np.eye(self.A.shape[0]) # self.A.shape[0] --> Rows
        # np.eye --> return a 2-D array with ones on the diagonal and zeros elsewhere
        for i in range(self.A.shape[0]):
            for j in range(self.A.shape[1]): # self.A.shape[1] --> Columns
                try:
                    aux[i,j] = self.A[i,j].subs(symbol_values).evalf()
                except:
                    pass
        return SE3_extended(aux) # Se entrega aux como una instancia de la clase SE3_extended

# Uso: objeto.symbolReplace([(simbolo1, valor_simbolo1),(simboloN, valor_simboloN)])

In [8]:
# Con la tabla de parámetros DH ya completada, se arman cada una de las MTH:
A01 = SE3_extended.Rz(q1  - radians(90)) * SE3_extended.Tz(l1) * SE3_extended.Tx(0) * SE3_extended.Rx(radians(90))
A12 = SE3_extended.Rz(q2) * SE3_extended.Tz(0) * SE3_extended.Tx(l2) * SE3_extended.Rx(0)
A23 = SE3_extended.Rz(q3  - radians(126)) * SE3_extended.Tz(0) * SE3_extended.Tx(l3) * SE3_extended.Rx(0)
A34 = SE3_extended.Rz(q4) * SE3_extended.Tz(0) * SE3_extended.Tx(l4) * SE3_extended.Rx(0)
A45 = SE3_extended.Ty(-l5) # Se traslada hacia la punta del marcador
T = A01 * A12 * A23 * A34 * A45


print('A01:')
A01.print()

print('A12:')
A12.print()

print('A23:')
A23.print()

print('A34:')
A34.print()

print('A45:')
A45.print()

print('T:')
T.print()

# Se adaptaron los cálculos de cinemática directa para que se puedan usar los servomotores MG996, los cuales solo recorren 180°
# Lo que se hizo fue desplazar cada uno de los 0° para que los movimientos sean compatibles con los servomotores
# Esto es importante porque si, por ejemplo, el servomotor tiene que retroceder 20° pero físicamente está en la posición de 0°, no podría realizar esa acción porque está por fuera de su rango de 0° a 180°
# Por ese motivo, es clave posicionarlos de forma adecuada para que el brazo pueda llegar a todos los movimientos solicitados

# La posición de 0° para cada q corresponde al segundo dibujo de cinemática directa que hice para el brazo
# A q1 se le resto -90° para que el barrido de 180° ocupe toda la parte frontal
# A q2 no es necesario modificarlo porque el rango de los 180° es el indicado. Si se lo baja más en sentido horario, el brazo podría chocar contra la base
# A q3 se le restó 126° para que el barrido de 180° comience en 0° en la posición de home para así poder aprovechar todo el intervalo
# A q4 no es necesario modificarlo porque el rango de grados es el adecuado para el servomotor

# El 126° para q3 sale de los 36° que obtuvimos para acomodar bien el brazo en los 0° que queríamos

A01:
  1.0*cos(q1 - 1.5707963267949) -6.12323399573677e-17*sin(q1 - 1.5707963267949) 1.0*sin(q1 - 1.5707963267949) 0             
  1.0*sin(q1 - 1.5707963267949) 6.12323399573677e-17*cos(q1 - 1.5707963267949) -1.0*cos(q1 - 1.5707963267949) 0             
  0            1.00000000000000 6.12323399573677e-17 1.0*l1        
  0            0            0            1.00000000000000  

A12:
  1.0*cos(q2)  -1.0*sin(q2) 0            1.0*l2*cos(q2)  
  1.0*sin(q2)  1.0*cos(q2)  0            1.0*l2*sin(q2)  
  0            0            1.00000000000000 0             
  0            0            0            1.00000000000000  

A23:
  1.0*cos(q3 - 2.19911485751286) -1.0*sin(q3 - 2.19911485751286) 0            1.0*l3*cos(q3 - 2.19911485751286)  
  1.0*sin(q3 - 2.19911485751286) 1.0*cos(q3 - 2.19911485751286) 0            1.0*l3*sin(q3 - 2.19911485751286)  
  0            0            1.00000000000000 0             
  0            0            0            1.00000000000000  

A34:
  1.0*cos(q4)  -

In [9]:
# Se les dan valores a las variables simbólicas:
A01_eval = A01.symbolReplace([(q1,radians(90.0)),(l1,14.085)])
A12_eval = A12.symbolReplace([(q2,radians(80.0)),(l2,12.725)])
A23_eval = A23.symbolReplace([(q3,radians(0.0)),(l3,10.222)])
A34_eval = A34.symbolReplace([(q4,radians(120.0)),(l4,11.200)])
A45_eval = A45.symbolReplace([(l5,7.400)])

# Posición del brazo en home para la configuración actual:
# q1: 90° (el brazo queda recto en la mitad de la pizarra)
# q2: 80° (el primer eslabón del brazo queda vertical)
# q3: 0°
# q4: 120° (donde el ángulo de pitch va a dar 0° por el q3 seteado)

In [10]:
S0 = SE3_extended()
S1 = S0 * A01_eval
S2 = S1 * A12_eval
S3 = S2 * A23_eval
S4 = S3 * A34_eval
S5 = S4 * A45_eval

print('S0:')
print(S0)

print('S1:')
print(S1)

print('S2:')
print(S2)

print('S3:')
print(S3)

print('S4:')
print(S4)

print('S5:')
print(S5)

S0:
   1         0         0         0         
   0         1         0         0         
   0         0         1         0         
   0         0         0         1         

S1:
   1         0         0         0         
   0         0        -1         0         
   0         1         0         14.09     
   0         0         0         1         

S2:
   0.1736   -0.9848    0         2.21      
   0         0        -1         0         
   0.9848    0.1736    0         26.62     
   0         0         0         1         

S3:
   0.6947    0.7193    0         9.31      
   0         0        -1         0         
  -0.7193    0.6947    0         19.26     
   0         0         0         1         

S4:
   0.2756   -0.9613    0         12.4      
   0         0        -1         0         
   0.9613    0.2756    0         30.03     
   0         0         0         1         

S5:
   0.2756   -0.9613    0         19.51     
   0         0        -1         0         
   

In [16]:
# https://bdaiinstitute.github.io/spatialmath-python/func_3d.html#spatialmath.base.transforms3d.trplot

limit = 50

trplot(S0.A, frame='0', dims = [0, limit, -limit, limit, 0, limit], length = 5, color  = 'red')
trplot(S1.A, frame='1', dims = [0, limit, -limit, limit, 0, limit], length = 5, color  = 'green')
trplot(S2.A, frame='2', dims = [0, limit, -limit, limit, 0, limit], length = 5, color  = 'black')
trplot(S3.A, frame='3', dims = [0, limit, -limit, limit, 0, limit], length = 5, color  = 'blue')
trplot(S4.A, frame='4', dims = [0, limit, -limit, limit, 0, limit], length = 5, color = 'orange')
trplot(S5.A, frame='5', dims = [0, limit, -limit, limit, 0, limit], length = 5, color = 'violet')

<Axes3D: xlabel='X', ylabel='Y', zlabel='Z'>

In [17]:
# Pose del efector final para las configuraciones articulares planteadas:
print('Matriz de transformación homogénea de {S5} respecto del sistema {S0}:')
print(S5)

Matriz de transformación homogénea de {S5} respecto del sistema {S0}:
   0.2756   -0.9613    0         19.51     
   0         0        -1         0         
   0.9613    0.2756    0         27.99     
   0         0         0         1         



In [18]:
# De la MTH T obtengo el vector de posición p que va desde la base {S0} hasta la punta del marcador marcador {S4}:
# Considerando que se logrará posicionar el gripper con las tres primeras variables articulares (q1,q2 y q3)

p = T.A[:3,3]
px, py, pz = p

p_values = S5.A[:3,3]
px_value, py_value, pz_value = p_values

print('Posición del efector final respecto de la terna base:')
print(f'- Px = {round(px_value,2)}')
print(f'- Py = {round(py_value,2)}')
print(f'- Pz = {round(pz_value,2)}')

Posición del efector final respecto de la terna base:
- Px = 19.51
- Py = 0.0
- Pz = 27.99


In [19]:
# Con la última variable articular q4 se controla la orientación del efector final
# En este caso, solo será posible manipular el ángulo de cabeceo (pitch) del robot

# Por lo calculado en cinemática inversa, si el ángulo de cabeceo es 0, q4 tendrá un valor que mantendrá al efector final paralelo a la superficie
# En caso contrario, el ángulo de cabeceo aportará en el cálculo de q4 para lograr la orientación deseada

# Para obtener el ángulo de cabeceo es necesario convertir la matriz de rotación de {S0} a {S5} en ángulos de Euler
# La composición de rotaciones seleccionada para los ángulos de Euler es XYZ (primero se rota en x, luego en y y finalmente en z)
# Teniendo en cuenta que cada una de las rotaciones es realizada respecto a los ejes de la terna base {S0} (sistema fijo)

# https://eecs.qmul.ac.uk/~gslabaugh/publications/euler.pdf
if S5.A[2,0] != 1 and S5.A[2,0] != -1:
    euler_angle_y = -sp.asin(S5.A[2,0])
    # euler_angle_y_sol2 = sp.pi - euler_angle_y_sol1
    
    euler_angle_x = sp.atan2(S5.A[2,1]/sp.cos(euler_angle_y), S5.A[2,2]/sp.cos(euler_angle_y))
    # euler_angle_x_sol2 = sp.atan2(S5.A[2,1]/sp.cos(euler_angle_y_sol2), S5.A[2,2]/sp.cos(euler_angle_y_sol2))
    
    euler_angle_z = sp.atan2(S5.A[1,0]/sp.cos(euler_angle_y), S5.A[0,0]/sp.cos(euler_angle_y))
    # euler_angle_z_sol2 = sp.atan2(S5.A[1,0]/sp.cos(euler_angle_y_sol2), S5.A[0,0]/sp.cos(euler_euler_angle_y_sol2))
else:
    euler_angle_z = 0
    
    if S5.A[2,0] == -1:
        euler_angle_y = sp.pi/2
        euler_angle_x = euler_angle_z + sp.atan2(S5.A[0,1], S5.A[0,2])
    else:
        euler_angle_y = -sp.pi/2
        euler_angle_x = -euler_angle_z + sp.atan2(-S5.A[0,1], -S5.A[0,2])

euler_angle_x = degrees(euler_angle_x)
euler_angle_y = degrees(euler_angle_y)
euler_angle_z = degrees(euler_angle_z)

print('Ángulos de Euler en grados (respecto a los vectores de la terna base) para orientar al efector final:')
print(f'Ángulo respecto X0: {round(euler_angle_x,2)}°')
print(f'Ángulo respecto Y0 (ángulo de interés): {round(euler_angle_y,2)}°') # Es la única orientación que podemos controlar con nuestro brazo robótico
print(f'Ángulo respecto Z0: {round(euler_angle_z,2)}°')

Ángulos de Euler en grados (respecto a los vectores de la terna base) para orientar al efector final:
Ángulo respecto X0: 90.0°
Ángulo respecto Y0 (ángulo de interés): -74.0°
Ángulo respecto Z0: 0.0°
